In [1]:
import pandas as pd
import torch
import torch.optim as optim
import numpy as np
import itertools

from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

g:\Code\Python\GitHub\maboo\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_4 = pd.read_csv('G:/Code/Python/GitHub/maboo/Brand/brand_data/brand_data_4.csv')
data_1 = pd.read_csv('G:/Code/Python/GitHub/maboo/Brand/brand_data/brand_data_1.csv')
data_1 = data_1.iloc[:, :2]
valid_brand = [i for i in set(data_1['brand'].to_list())]

data = pd.concat([data_1, data_4])
data.reset_index(drop=True, inplace=True)

valid_index = []
for i in range(len(data)):
    if data['brand'][i] in valid_brand:
        data['brand'][i] = 1
        valid_index.append(i)
    else:
        data['brand'][i] = 0

# X = data['name']
# y = le.fit_transform(data['bra_整合'])
# X_train, X_val, y_train, y_val = train_test_split(data['name'].values, data['brand'].values, test_size=0.2, random_state=89)

In [3]:
X_train = data[data['brand'] == 1]['name']
y_train = data[data['brand'] == 1]['brand']

X_valid = data[data['brand'] == 0]['name']
y_valid = data[data['brand'] == 0]['brand']

In [4]:
from data_preprocess import BrandDataset
from cvdd_net import CVDDNet

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
train_dataset = BrandDataset(X_train, y_train, tokenizer, 512)
valid_dataset = BrandDataset(X_valid, y_valid, tokenizer, 512)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [7]:
net = CVDDNet('bert-base-multilingual-uncased', 100, 3)
ids = train_dataset[1]['ids'].view([1, -1])
mask = train_dataset[1]['mask'].view([1, -1])

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
ids.shape

torch.Size([1, 512])

In [9]:
cosine_dists, context_weight, A = net(ids.to(torch.long), mask.to(torch.long))

In [10]:
cosine_dists

tensor([[0.5264, 0.5128, 0.4957]], grad_fn=<MulBackward0>)

In [12]:
context_weight

tensor([[0.3333, 0.3333, 0.3333]], grad_fn=<SoftmaxBackward0>)

In [14]:
A.shape

torch.Size([1, 3, 512])